In [1]:
# Importar las librerias necesarias
import psycopg2
import requests 
import pandas as pd
import lxml
import html5lib
import re
from bs4 import BeautifulSoup 
from sqlalchemy import create_engine

In [2]:
import load_dotenv
from dotenv import dotenv_values
import os

In [3]:
# Seteo de las claves para el acceso
env_vars = dotenv_values(".env")
db_host = env_vars['host']
db_port = env_vars['port']
db_name = env_vars['database']
db_user = env_vars['user']
db_password = env_vars['password']

In [4]:
'''Funcion para obtener la tabla de wikipedia'''
def get_df(url):
    # Get url
    table_class="wikitable sortable jquery-tablesorter"
    response=requests.get(url)
    # Parse the response 
    soup = BeautifulSoup(response.text, 'html.parser')
    soup_tabla=soup.find('table',{'class':"wikitable"})
    df=pd.read_html(str(soup_tabla), flavor= 'html5lib')
    df=pd.DataFrame(df[0])
    return df

In [5]:
'''DF de paises'''
df = get_df('https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes')

In [6]:
'''Preprocesamiento del df previo a ser exportado a postgres'''
df.columns = df.columns.get_level_values(1)
df.columns = df.columns.str.replace(r'\[[^\]]*\]', '', regex=True).str.replace(r'\s', '_', regex=True).str.lower()
df['country_name'] = df['country_name'].apply(lambda x: re.sub(r'\[[^\]]*\]|\([^)]*\)', '', x).strip())
df.head()

,country_name,official_state_name,sovereignty,alpha-2_code,alpha-3_code,numeric_code,subdivision_code_links,internet_cctld
0,Afghanistan,The Islamic Republic of Afghanistan,UN member state,.mw-parser-output .monospaced{font-family:mono...,AFG,004,ISO 3166-2:AF,.af
1,Åland Islands,Åland,Finland,AX,ALA,248,ISO 3166-2:AX,.ax
2,Albania,The Republic of Albania,UN member state,AL,ALB,008,ISO 3166-2:AL,.al
3,Algeria,The People's Democratic Republic of Algeria,UN member state,DZ,DZA,012,ISO 3166-2:DZ,.dz
4,American Samoa,The Territory of American Samoa,United States,AS,ASM,016,ISO 3166-2:AS,.as


In [7]:
'''DF del continente Americano'''
america = get_df('https://en.wikipedia.org/wiki/Americas')


In [8]:
'''Preprocesamiento del df previo a ser exportado a postgres'''
america.columns = america.columns.str.replace(r'\[[^\]]*\]', '', regex=True).str.lower().str.strip().str.replace(r'\s', '_', regex=True)
america['country_or_territory'] = america['country_or_territory'].apply(lambda x: re.sub(r'\[[^\]]*\]|\([^)]*\)', '', x).strip())
america.rename(columns = {'common_languages_(official_in_bold)':'common_languages', 'country_or_territory': 'country_name'}, inplace = True)
america= america.applymap(lambda x: re.sub(r'\[[^\]]*\]|\([^)]*\)', '', str(x)).strip())
world = df[['country_name', 'alpha-3_code']]
america_2 = america.merge(world, how= 'left', left_on = 'country_name', right_on = 'country_name')
america_2 = america_2.dropna(subset=['alpha-3_code'])
america_2['population'] = america_2['population'].apply(lambda x: x.replace(',', '.'))
america_2[['total_area_(km2)', 'population', 'pop._density_(per_km2)']] = america_2[['total_area_(km2)', 'population', 'pop._density_(per_km2)']].apply(pd.to_numeric)
america_2.head()

,country_name,total_area_(km2),population,pop._density_(per_km2),common_languages,capital,alpha-3_code
0,Anguilla,91,13452.0,164.8,English,The Valley,AIA
1,Antigua and Barbuda,442,86295.0,199.1,"Creole, English",St. John's,ATG
2,Argentina,2766890,42669500.0,14.3,Spanish,Buenos Aires,ARG
3,Aruba,180,101484.0,594.4,"Papiamentu, Spanish, Dutch",Oranjestad,ABW
5,Barbados,430,285000.0,595.3,"Bajan, English",Bridgetown,BRB


In [9]:
'''Se corrobora la dimension de los df creados'''
america.shape, df.shape, america_2.shape

((62, 6), (271, 8), (52, 7))

In [10]:
'''Conexion con pgadmin'''
conn = psycopg2.connect (
    host=db_host,
    port=db_port,
    database=db_name,
    user=db_user,
    password=db_password
)
cursor = conn.cursor()
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}")
'''Se exportan los df'''
df.to_sql(name = 'paises', con = engine, if_exists='replace', index=False)
america_2.to_sql(name = 'america', con = engine, if_exists='replace', index=False)


52

In [11]:
try:
    # Define el cambio
    updating = "UPDATE america SET country_name = %s, population = %s WHERE country_name = %s"
    # Valores a cambiar
    new_value_ar = "Republica Argentina"
    prior_value_ar = "Argentina"
    new_population_ar =  46420985
    new_value_uy = "Republica Oriental del Uruguay"
    prior_value_uy = "Uruguay"
    new_population_uy =  3509274
    # Ejecuta el cambio
    cursor.execute(updating, (new_value_ar, new_population_ar, prior_value_ar))
    cursor.execute(updating, (new_value_uy, new_population_uy, prior_value_uy))
    # Commit 
    conn.commit()
    print("Update operation successful.")

except Exception as e:
    # Se usa rollback, en el caso de que ocurra algun error
    conn.rollback()
    print("Update operation failed. Error:", str(e))

finally:
    # Restaura el autocommit, y se cierra conexion
    conn.autocommit = True
    cursor.close()
    conn.close()

Update operation successful.
